In [ ]:
import numpy as np
import tensorflow as tf


# SIEGE Model
class SIEGEModel(tf.keras.Model):
    def __init__(self, input_dim, output_dim):
        super(SIEGEModel, self).__init__()
        # 定义模型的各层 Define the layers of the model
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, output_dim)
        self.graph_conv_layer1 = GraphConvolution(output_dim, activation='relu')
        self.graph_conv_layer2 = GraphConvolution(output_dim, activation='relu')
        self.pooling_layer = tf.keras.layers.GlobalAveragePooling1D()
        self.fc_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        # 定义模型的前向传播 Define forward propagation of the model
        x = self.embedding_layer(inputs)
        x = self.graph_conv_layer1(x)
        x = self.graph_conv_layer2(x)
        x = self.pooling_layer(x)
        x = self.fc_layer(x)
        return x


# 定义自监督预训练任务类 Define a self-supervised pretraining task class
class SelfSupervisedTask(tf.keras.Model):
    def __init__(self, input_dim, output_dim):
        super(SelfSupervisedTask, self).__init__()
        # 定义自监督任务的各层 Define the layers of a self-supervised task
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, output_dim)
        self.conv_layer = tf.keras.layers.Conv1D(output_dim, kernel_size=3, padding='same', activation='relu')
        self.pooling_layer = tf.keras.layers.MaxPooling1D(pool_size=2)
        self.fc_layer1 = tf.keras.layers.Dense(output_dim, activation='relu')
        self.fc_layer2 = tf.keras.layers.Dense(output_dim, activation='relu')

    def call(self, inputs):
        # 定义自监督任务的前向传播 Define forward propagation for self-monitoring tasks
        x = self.embedding_layer(inputs)
        x = self.conv_layer(x)
        x = self.pooling_layer(x)
        x = self.fc_layer1(x)
        x = self.fc_layer2(x)
        return x


# 初始化SIEGE模型和自监督预训练任务 Initialize the SIEGO model and the self-supervised pre-training task
sieve_model = SIEGEModel(input_dim, output_dim)
self_supervised_task = SelfSupervisedTask(input_dim, output_dim)

# 循环遍历训练数据，进行自监督预训练 Loop through the training data for self-supervised pre-training
for epoch in range(num_epochs):
    for batch in train_data:
        with tf.GradientTape() as tape:
            # 获取输入数据 input data
            inputs = batch['inputs']
            labels = batch['labels']

            # 进行自监督预训练 self-supervised pre-training
            self_supervised_outputs = self_supervised_task(inputs)
            # 计算预训练任务的损失 Loss of pre-training tasks
            self_supervised_loss = tf.losses.mean_squared_error(inputs, self_supervised_outputs)

        # 更新自监督预训练任务的参数 Update the parameters of a self-supervised pre-training task
        grads = tape.gradient(self_supervised_loss, self_supervised_task.trainable_variables)
        optimizer.apply_gradients(zip(grads, self_supervised_task.trainable_variables))

# 循环遍历训练数据，进行SIEGE模型的训练 Loop through the training data to train the SIEGO model
for epoch in range(num_epochs):
    for batch in train_data:
        with tf.GradientTape() as tape:
        
        inputs = batch['inputs']
        labels = batch['labels']
        # 使用自监督预训练任务得到的特征进行SIEGE模型的前向传播 The features obtained by the self-supervised pre-training task are used to carry out forward propagation of the SIEGO model
        features = self_supervised_task(inputs)
        outputs = sieve_model(features)

        # 计算SIEGE模型的损失 the loss of the SIEGO model
        loss = tf.losses.binary_crossentropy(labels, outputs)

        # 更新SIEGE模型的参数 Update the parameters of the SIEGE model
    grads = tape.gradient(loss, sieve_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, sieve_model.trainable_variables))

# 在验证集上进行模型评估 Model evaluation on validation sets
accuracy = evaluate(sieve_model, val_data)
print(f'Epoch {epoch + 1}, Accuracy: {accuracy}')

# 在测试集上进行模型评估 Model evaluation on the test set
test_accuracy = evaluate(sieve_model, test_data)
print(f'Test Accuracy: {test_accuracy}')

# 使用训练好的SIEGE模型进行预测 Predictions are made using the trained SIEGO model
predictions = sieve_model.predict(test_inputs)